### *Imports necesarios para el desarrollo del proyecto*

In [1]:
import io
import joblib
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from flask import Flask, request, jsonify
from keras.models import load_model
from keras.layers import InputLayer
from keras.utils import load_img, img_to_array

In [2]:
model = load_model('cnn_model.keras')

### *Creación de las funciones necesarias*

In [3]:
class RGBError(ValueError):  #Error personalizado para indicar que la imagen no esta en modo RGB
    def __init__(self, message='The image have to be in RGB mode. Please modificate it and try to send it again.'):
        self.message = message
        super().__init__(self.message)


def preprocess_img(img_file, size=224):  # Función para la carga y procesado de las imágenes

    if hasattr(img_file, 'read'):
        img_bytes = img_file.read()
        img_file = io.BytesIO(img_bytes)
    
    img = load_img(img_file, target_size=(size, size))

    img_array = img_to_array(img)

    if img_array.shape[-1] != 3:
        raise RGBError()
    
    img_norm = img_array.astype(np.float32)/255.0
    img_expanded = np.expand_dims(img_norm, axis=0)
    
    return img_expanded

### *Creación de la API*

In [4]:
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'No image was found in your request.'}), 400

    file = request.files['image']
    
    if file.filename == '':
        return jsonify({'error': 'No file was selected.'}), 400

    try:        
        prep_img = preprocess_img(file)

        pred = tf.argmax(model.predict(prep_img), axis=-1)

        response = {'prediction': int(pred)}
        return jsonify(response)

    except RGBError as e:
        return jsonify({'error': str(e)}), 400
        
    except Exception as e:
        return jsonify({'error': f'Internal server error: {str(e)}'}), 500

In [5]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


127.0.0.1 - - [09/Aug/2025 20:04:56] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step


...

127.0.0.1 - - [09/Aug/2025 20:04:58] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step


127.0.0.1 - - [09/Aug/2025 20:04:58] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step


127.0.0.1 - - [09/Aug/2025 20:04:58] "POST /predict HTTP/1.1" 200 -
